### How to read and plot IMERG data from NASA GES DISC and Earthdata using Python:

Data: https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python

How-to-Plot Guide (Jupyter Notebook): https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python

### Overview:

This recipe shows how to read data from the Global Precipitation Measurement (GPM) mission's IMERG dataset using Python.

### Example:

**Example data:** GPM Level 3 IMERG Monthly 0.1 x 0.1 degree Precipitation (GPM_3IMERGM) for January 2014.

**Estimated Time to complete the following procedures:** 20 minutes

### Prerequisites:

**Task:** Viewing Data

**Best When:** The user wants to read in GPM IMERG data using Python

**Requirements:** Python and the free packages: h5py, numpy, matplotlib, and cartopy. Matplotlib and Cartopy are only needed for plotting.

### Procedure:

#### 1. Download the data

Before accessing data at GES DISC, a user must first register with Earthdata Login, then be authorized to access data at GES DISC by following steps at: https://disc.gsfc.nasa.gov/information/documents?title=Data%20Access

1. In a web browser, go to: https://disc.gsfc.nasa.gov </ul>
2. In the Search field, enter "imerg" and press enter.
3. Click on the latest version of the half hourly/daily/monthly average IMERG data, currently version 7:

    i)   Click on the "Online Archive" button on the right.

    ii)  Click on the "2014/" folder
    
    iii) Click on the link "3B-MO.MS.MRG.3IMERG.20140101-S000000-E235959.01.V06B.HDF5" to download the data file.

**Note:** Instead of doing steps i) - iii) above, you could also just click on Subset/Get Data below the desired version and select your desired time range

**Note:** this recipe works for any IMERG data, not just monthly estimates.


#### 2. Run the following cells to learn how to read and plot IMERG data with Python

In [1]:
#Import the required Python libraries. If any of the following import commands fail, then
    #check the local Python environment and install any missing packages.

import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker


In [2]:
#Open the IMERG data for reading

#Add a path to the filename if it the IMERG file is not in the working directory
file_date = '20210824'
day_folder = os.path.join(os.getcwd(), file_date)
fn = os.path.join(day_folder, 'IMERG_files', '3B-HHR.MS.MRG.3IMERG.20210824-S143000-E145959.0870.V07A.HDF5')
f = h5py.File(fn, 'r')


In [3]:
#View the available groups in the file and the variables in the 'Grid' group:

#groups = [ x for x in f.keys() ]
#print(groups)
print([ x for x in f.keys() ])

#gridMembers = [ x for x in f['Grid'] ]
#print(gridMembers)
print([ x for x in f['Grid'] ])                #see IMERG_V07_ATBD_final.pdf p.32 for what these variables mean
print([ x for x in f['Grid/Intermediate'] ])   #see IMERG_V07_ATBD_final.pdf p.32 for what these variables mean
                
                                               #see IMERG_V07_ATBD_final.pdf p.48-49 for Quality Index information

['Grid']
['Intermediate', 'nv', 'lonv', 'latv', 'time', 'lon', 'lat', 'time_bnds', 'lon_bnds', 'lat_bnds', 'precipitation', 'randomError', 'probabilityLiquidPrecipitation', 'precipitationQualityIndex']
['MWprecipitation', 'MWprecipSource', 'MWobservationTime', 'IRprecipitation', 'IRinfluence', 'precipitationUncal']


In [4]:
#Read the precipitation, latitude, and longitude data:

#Get the precipitation, latitude, and longitude variables
print ('Time Shape:', f['Grid/time'].shape)
print ('Original Precip Shape:', f['Grid/precipitation'].shape)

precip = f['Grid/precipitation'][0][:][:]
print ('New Precip Shape:', precip.shape)

precip = np.transpose(precip)
print ('Newest Precip Shape:', precip.shape)

#mask blank data
precip_masked = np.ma.masked_where(precip < 0, precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
precip_masked = np.ma.masked_where(np.isnan(precip_masked), precip_masked)  #masks NaN values (not masked in previous line)

theLats = f['Grid/lat'][:]
theLons = f['Grid/lon'][:]
print ('Latitude Shape:', theLats.shape)
print ('Longitude Shape:', theLons.shape)

x, y = np.meshgrid(theLons, theLats)
print ('Longitude grid shape:', x.shape)
print ('Latitude grid shape:', y.shape)

Time Shape: (1,)
Original Precip Shape: (1, 3600, 1800)
New Precip Shape: (3600, 1800)
Newest Precip Shape: (1800, 3600)
Latitude Shape: (1800,)
Longitude Shape: (3600,)
Longitude grid shape: (1800, 3600)
Latitude grid shape: (1800, 3600)


In [5]:
print (precip.min())
print (precip_masked.min())
print (precip.max())
print (precip_masked.max())

-9999.9
0.0
54.55
54.55


In [6]:
#Plot the data using matplotlib and Cartopy

# Set the figure size, projection, and extent
fig = plt.figure(figsize = (21,7))
ax = plt.axes(projection = ccrs.PlateCarree())
ax.set_extent([-180, 180, -60, 60], ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

# Add coastlines and formatted gridlines
ax.coastlines(resolution = '110m', linewidth = 1)
gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 1, color = 'k', linestyle = '--')
gl.top_labels = False
gl.right_labels = False
#gl.xlines = False
gl.xlocator = mticker.FixedLocator([-180, -90, 0, 90, 180])
gl.ylocator = mticker.FixedLocator([-60, -50, -25, 0, 25, 50, 60])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':16, 'color':'black'}
gl.ylabel_style = {'size':16, 'color':'black'}

# Set contour levels and draw the plot
#clevs = np.arange(0.1, 40.5, 1)                             #for linear colorbar
clevs = np.logspace(np.log10(0.1), np.log10(40), num = 40)   #for logarithmic colorbar
plt.pcolormesh(x, y, precip_masked, norm = mplc.LogNorm(vmin = 0.1, vmax = 40), cmap = plt.cm.jet, transform = ccrs.PlateCarree())
#plt.contourf(x, y, precip_masked, clevs, norm = 'log', extend = 'max', cmap = plt.cm.jet, transform = ccrs.PlateCarree())
plt.title('GPM IMERG Half Hourly Mean Rain Rate for 2021/08/24 15Z', size = 24)

# Set colorbar
ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40])
cb = plt.colorbar(ax = ax, orientation = "vertical", ticks = ticks_imerg, pad = 0.02, aspect = 16, shrink = 0.8)
cb.set_label('IMERG [mm hr$^{-1}$]', size = 20)
cb.ax.tick_params(labelsize = 16)
cb.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
cb.ax.yaxis.set_ticks_position('right')
cb.ax.yaxis.set_label_position('right')

#Save the figure as a PNG:
plt.savefig('/Users/brodenkirch/Desktop/GPM_IMERGV7_plot.png', bbox_inches = 'tight', pad_inches = 0.1)
plt.close()


In [7]:
np.logspace(np.log10(0.1), np.log10(40), num = 40)

array([ 0.1       ,  0.11660562,  0.13596871,  0.15854716,  0.1848749 ,
        0.21557452,  0.25137201,  0.2931139 ,  0.34178728,  0.39854318,
        0.46472375,  0.54189402,  0.63187889,  0.7368063 ,  0.85915756,
        1.00182601,  1.16818545,  1.3621699 ,  1.58836667,  1.85212482,
        2.15968165,  2.51831021,  2.93649126,  3.42411388,  3.99270926,
        4.65572344,  5.42883523,  6.33032705,  7.38151718,  8.60726396,
       10.03655361, 11.70318568, 13.64657237, 15.91267048, 18.55506827,
       21.63625262, 25.22908677, 29.41853335, 34.30366357, 40.        ])